<a href="https://colab.research.google.com/github/Jeidsgn/Frame_overlay/blob/main/Frame_overlay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conectarse con Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

En *'directorio'* se ubica la cerpeta donde se encuentran las carpetas resultantes de EbSynth.
En *'resultado'* se escribe el nombre dela carpeta donde estará el resultado de la superposición. No es necesario que esté creada, pues se creará automaticamente.

In [ ]:
directorio = '/content/drive/MyDrive/backup/Docu/Diseño/anime/Experimen2/filtro'
resultado = directorio + '/Overlay'


carpetas = os.listdir(directorio)
#Crear carpeta resultado si no existe
if not os.path.exists(resultado):
  os.makedirs(resultado)
else:
  # Obtenemos la lista de archivos de la carpeta
  archivos = os.listdir(resultado)
  # Recorremos cada archivo y lo eliminamos
  for archivo in archivos:
    os.remove(resultado + '/' + archivo)

In [ ]:
def imprimir_porcentaje(iteracion, total_fotogramas):
    porcentaje = (iteracion / total_fotogramas) * 100
    print("\rProgreso: {:.2f}%".format(porcentaje), end="")

frames = []
for carpeta in carpetas:
    if carpeta.startswith("out_"):
        fotogramas_carpeta = os.listdir(os.path.join(directorio,carpeta))
        frames.extend(fotogramas_carpeta)
f = len(frames)

In [ ]:
carpetas = os.listdir(directorio)
i=0
carpetas.sort()
for carpeta in carpetas:
  # Verificamos si la carpeta cumple con el patrón "out_[#####]"
  if 'out_' in carpeta:
    # Obtenemos el número del fotograma key
    numero = carpeta.split('_')[1]
    # Abrimos los fotogramas de la carpeta
    fotogramas = os.listdir(directorio + '/' + carpeta)
    # Abrimos el primer fotograma para obtener sus dimensiones
    img = cv2.imread(directorio + '/' + carpeta + '/' + fotogramas[0])
    # Obtenemos el menor y mayor fotograma, primero la lista de nombres
    numeros = []
    # Pasamos nombres a números
    for fotograma in fotogramas:
      nombre = int(fotograma.split('.')[0])
      numeros.append(nombre)
    # Obtenemos el menor y mayor fotograma
    numero_menor = min(numeros)
    numero_mayor = max(numeros)
    # Recorremos cada fotograma
    for fotograma in fotogramas:
         
      # Abrimos el fotograma
      img = cv2.imread(directorio + '/' + carpeta + '/' + fotograma)
      # revisamos si existe ya ese fotograma en resultados
      if os.path.exists(resultado + '/' + fotograma):
        # Si existe, usamos ese como base
        img_base = cv2.imread(resultado + '/' + fotograma)
      else:
        # si no, usamos el fotograma abierto
        img_base = img
      if int(numero)<int(fotograma.split('.')[0]):
        # Calculamos la distancia máxima entre el fotograma key y el fotograma mayor
        distancia_máxima = abs(numero_mayor - int(numero))
        # Calculamos la distancia del fotograma actual al fotograma key
        distancia_al_key = abs(int(numero) - int(fotograma.split('.')[0]))
        # Calculamos la opacidad de manera lineal
        opacidad = 1-(distancia_al_key / distancia_máxima)
      elif int(numero)>int(fotograma.split('.')[0]):
        # Calculamos la distancia máxima entre el fotograma key y el fotograma menor
        distancia_máxima = abs(numero_menor - int(numero))
        # Calculamos la distancia del fotograma actual al fotograma key
        distancia_al_key = abs(int(numero) - int(fotograma.split('.')[0]))
        # Calculamos la opacidad de manera lineal
        opacidad = 1-(distancia_al_key / (distancia_máxima))  
      else:
        opacidad = 1
      # Aplicamos la opacidad a la imagen
      img_resultado = cv2.addWeighted(img_base, 1-opacidad, img, opacidad, 0)
      
      #cv2_imshow(img_resultado)
      # Guardamos la imagen resultante en la carpeta resultados
      cv2.imwrite(resultado + '/' + fotograma, img_resultado)
      # Porcentaje
      i += 1
      imprimir_porcentaje(i, f)

print('Proceso terminado')